In [ ]:
import pickle
import json
import mmcv
from tqdm import tqdm_notebook as tqdm
from mmdet.datasets import build_dataloader, build_dataset
from pycocotools.coco import COCO
import pycocotools.coco as cocoapi
import pycocotools.mask as MASK
import os
import cv2
from icecream import ic
import matplotlib.pyplot as plt
from mmdet.models import build_detector
from mmcv.runner import load_checkpoint, get_dist_info
import numpy as np
def get_original_size(name):
    datadir = "/home/xfr/rssid/data/test/images/"
    files = os.listdir(datadir)
    img_name = ""
    for file in files:
        img_name = file.split(".")[0]
        if(img_name == name):
            break
    filename = os.path.join(datadir, file)
    img = cv2.imread(filename)
    h, w, _ = img.shape
    return h,w
cfg = mmcv.Config.fromfile("configs/rs_cascade_mask_rcnn_r50_fpn_ohem.py")
CLASSES = ('tennis-court', 'container-crane', 'storage-tank', 'baseball-diamond', 'plane','ground-track-field',
               'helicopter', 'airport','harbor', 'ship', 'large-vehicle', 'swimming-pool', 'soccer-ball-field',
               'roundabout', 'basketball-court', 'bridge', 'small-vehicle', 'helipad')
results = mmcv.load("./result/test.pkl")
dataset = build_dataset(cfg.data.test)
img_infos = dataset.load_annotations("/home/xfr/mmdetection/data/rscup/annotation/annos_rscup_test.json")
ann = {}
last_name = ""
last = ""
for i in tqdm(range(len(results))):
    filename = img_infos[i]['filename']
    items = filename.split("_")
    name = items[0]
    if(name == last_name):
        H, W = last
    else:
        H, W = get_original_size(name)
    last_name = name
    last = (H, W)
    if name not in ann:
        ann[name] = {"bbox": [[] for i in range(len(dataset.CLASSES))], "seg": [[] for i in range(len(dataset.CLASSES))],
                     "vis": [[] for i in range(len(dataset.CLASSES))]}
    scale_factor = 1/float(items[1])
    locx = int(float(items[2]))
    locy = int(float(items[3]))
    result = results[i]
    bboxes = result[0]
    segs = result[1]
    for cls in range(len(dataset.CLASSES)):
        bbox_cls = bboxes[cls]
        seg_cls = segs[cls]
        if( len(bbox_cls) > 0):
            for bbox, rle in zip(bbox_cls, seg_cls):
                xmin, ymin, xmax, ymax, score = bbox
                xmin += locx
                ymin += locy
                xmax += locx
                ymax += locy
                bounding_box = np.array([xmin*scale_factor, ymin*scale_factor, xmax*scale_factor, ymax*scale_factor, score])
                mask = MASK.decode(rle)
                contours, hierarchy = cv2.findContours((mask).astype(np.uint8), cv2.RETR_TREE,
                                                                 cv2.CHAIN_APPROX_SIMPLE)
                areas = [cv2.contourArea(contour)  for contour in contours]
                if(len(areas) > 0):
                    index = np.argmax(areas)
                    contour = contours[index]
                    rect = cv2.minAreaRect(contour)
                    box = cv2.boxPoints(rect)
                    box += np.array([locx, locy])
                else:
                    continue
                ann[name]["bbox"][cls].append(bounding_box)
                ann[name]['vis'][cls].append(np.array([box[0]*scale_factor, box[1]*scale_factor, box[2]*scale_factor, box[3]*scale_factor]))
                poly = []
                for point in box:
                    poly.append(point[0])
                    poly.append(point[1])
                poly = np.array(poly, np.double)
                poly = [poly*scale_factor]
                rles = MASK.frPyObjects(poly, H, W)
                rle = MASK.merge(rles)
                ann[name]["seg"][cls].append(rle)
print(len(ann.keys()))
mmcv.dump(ann, "./result/test_temp.pkl") 

loading annotations into memory...
Done (t=0.27s)
creating index...
index created!
loading annotations into memory...
Done (t=0.24s)
creating index...
index created!


In [4]:
from mmdet.ops.nms import nms_wrapper
def nms(ann, thresh):
    for name in tqdm(ann.keys()):
        info = ann[name]
        for cls in range(len(dataset.CLASSES)):
            bbox = np.array(info['bbox'][cls], np.float32)
            seg = np.array(info['seg'][cls])
            vis = np.array(info['vis'][cls])
            if(len(bbox)<=0):
                continue
            h, w = seg[0]['size']
            #print(bbox.shape)
            _, inds = nms_wrapper.nms(bbox, thresh)
           # print(len(inds))
            ann[name]['bbox'][cls] = bbox[inds]
            ann[name]['seg'][cls] = seg[inds]
            ann[name]['vis'][cls] = vis[inds]
    return ann
thresh=0.5
# result = copy.deepcopy(ann)
ann = mmcv.load("./result/test_temp.pkl")
result2 = nms(ann, thresh)
mmcv.dump(result2, "./result/test_postnms.pkl")

In [6]:
result2 = mmcv.load("./result/test_postnms.pkl")
names = list(result2.keys())
res = { CLASSES[cls]: [] for cls in range(len(dataset.CLASSES)) }
for name in tqdm(names):
    datadir = "/home/xfr/rssid/data/test/images/"
    files = os.listdir(datadir)
    img_name = ""
    for file in files:
        img_name = file.split(".")[0]
        if(img_name == name):
            break
    filename = os.path.join(datadir, file)
    img = cv2.imread(filename)
    result = result2[name]
    bboxes = result["bbox"]
    segs = result['vis']
    
    for cls in range(len(dataset.CLASSES)-1):
        curr_class = CLASSES[cls]
        bbox_cls = bboxes[cls]
        seg_cls = segs[cls]
        if( len(bbox_cls) > 0):
            for bbox, rle in zip(bbox_cls, seg_cls):
                rle = np.array(rle)
                xmin, ymin, xmax, ymax, score = bbox
                location = list(rle.flatten())
                location = [str(int(x)) for x in location]
                out = name + " "+str(score)+ " "+" ".join(location)
                res[curr_class].append(out)
                cv2.putText(img, CLASSES[cls], (int(xmin), int(ymin)), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
                cv2.polylines(img, np.array([rle], np.int32), 1, (0, 255, 0), 1)
                #cv2.poly(img, np.array([rle], np.int32), (0,))
    cv2.imwrite("./result/test/{}.jpg".format(name), img)
for key in res.keys():
    print(key)
    fp = open("/result/detection/"+key+".txt", 'w')
    print(len(res[key]))
    for line in res[key]:
        fp.write(line+"\n")
    fp.close()

tennis-court


FileNotFoundError: [Errno 2] No such file or directory: '/result/detection/tennis-court.txt'